In this project the main goal is to scrape popular websites for its job posting data.
And maybe use some sort of graphing library to look at market corrolations.

Helpful links.
https://requests.readthedocs.io/en/latest/
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [16]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import seaborn as sb
import time 
import sqlalchemy
import re
from random import randint 

In [2]:
#Creating pandas dataframe to store all the data
cols = ['Title', 'Company', 'CompanyRating', 'Location', 'Salary']
#Creating df with wanted columns
df = pd.DataFrame(columns=cols)

In [3]:
for x in range (2001): 
    #Target url including page number so its easier to get many pages worth of data
    url = 'https://www.simplyhired.com/search?l=New+York%2C+NY&pn='+str(x)+'&job=aj_3klHDEeiZGRtUn5-oMbsIOhWSCc_c_X4kLfL_1LUtY6aPhtgsyQ'
    #Format the URL
    req = requests.get(url).text
    #Read with BS4
    soup = BeautifulSoup(req, 'lxml')
    #Seperate each of the jobs by looking at the htlm tag in this case is seperated by  or hyperlink anchor tag
    #jobs = soup.find('a', class_="SerpJob-link card-link")
    #jobs = soup.find('a', {"class": "SerpJob-link card-link"})
    #Since there is no class in the tag we get all the lists
    #all_jobs = soup.find_all('li') 
    #Get all the job cards
    jobs = soup.find_all('article', {"class": "SerpJob"})
    #Further Parsing the data
    #Try statement incase some of the salaries or reviews are missing
    for job in jobs:
        try:
            title = job.find('a', {"class": "SerpJob-link card-link"}).text
        except:
            title = np.nan
        try:
            company = job.find('span', {"class": "JobPosting-labelWithIcon jobposting-company"}).text
        except:
            company= np.nan
        try:
            company_rating = job.find('span', {"class": "CompanyRatings-ratingDigit"}).text
        except:
            company_rating = np.nan
        try:
           location = job.find('span', {"class": "jobposting-location"}).text
        except:
            location = np.nan
        try:
            salary = job.find('div', {"class": "jobposting-salary SerpJob-salary"}).text
        except:
            salary = np.nan
        #For every job fill in a single row with its information if string is not found fill with Nan object and store inside of a dictionary
        nr = [{'Title': title, 'Company': company, 'CompanyRating': company_rating, 'Location': location, 'Salary':salary}]
        #Insert the dictionary in df
        df = df.append(nr, ignore_index=True)


In [4]:
df

,Title,Company,CompanyRating,Location,Salary
0,Warehouse Associate - Part Time (Days Shift),Lowe's,3.5,"New York, NY","$3,500 - $4,500 a month"
1,Teacher - Entry - Level to Experienced (Harlem...,Success Academy Charter Schools,NaN,"New York, NY",NaN
2,Part-Time Stocker,Aldi,3.3,"Brooklyn, NY",$19 an hour
3,Office Helper,CALL-A-HEAD CORP.,4.5,"Broad Channel, NY","$700 - $1,100 a week"
4,Office Helper,CALL-A-HEAD CORP.,4.5,"Broad Channel, NY","$700 - $1,100 a week"
...,...,...,...,...,...
16049,General Bookkeeper,"Algin Management Co., LLC",NaN,"New York, NY",NaN
16050,Barista in Training,Interlude Coffee & Tea,NaN,"New York, NY",NaN
16051,"Hollister Co. - Stock Associate, FIFTH AVENUE",Hollister Co. Stores,3.5,"New York, NY +8 locations",NaN
16052,Part Time Branch Ambassador- King's Plaza,Capital One,3.9,"Brooklyn, NY",NaN


In [5]:
#Saving Data to csv
#df.to_csv(r"C:\Users\Alif\Desktop\Python files\vscode_python\30-days-python\python_webscraping\raw_data_new.csv", index=False)

In [29]:
df = pd.read_csv(r"C:\Users\Alif\Desktop\Python files\vscode_python\30-days-python\python_webscraping\raw_data_new.csv")

In [6]:
df.drop_duplicates()

,Title,Company,CompanyRating,Location,Salary
0,Warehouse Associate - Part Time (Days Shift),Lowe's,3.5,"New York, NY","$3,500 - $4,500 a month"
1,Teacher - Entry - Level to Experienced (Harlem...,Success Academy Charter Schools,NaN,"New York, NY",NaN
2,Part-Time Stocker,Aldi,3.3,"Brooklyn, NY",$19 an hour
3,Office Helper,CALL-A-HEAD CORP.,4.5,"Broad Channel, NY","$700 - $1,100 a week"
5,Part Time Cashier Associates,Lowe's,3.5,"Brooklyn, NY","$2,500 - $3,500 a month"
...,...,...,...,...,...
15099,General Bookkeeper,"Algin Management Co., LLC",NaN,"New York, NY",NaN
15100,Barista in Training,Interlude Coffee & Tea,NaN,"New York, NY",NaN
15101,"Hollister Co. - Stock Associate, FIFTH AVENUE",Hollister Co. Stores,3.5,"New York, NY +8 locations",NaN
15102,Part Time Branch Ambassador- King's Plaza,Capital One,3.9,"Brooklyn, NY",NaN


In [20]:
df.duplicated().sum()

14172

In [21]:
df.duplicated().value_counts()

True     14172
False     1882
dtype: int64

In [30]:
df = df.drop_duplicates()

In [8]:
df

,Title,Company,CompanyRating,Location,Salary
0,Warehouse Associate - Part Time (Days Shift),Lowe's,3.5,"New York, NY","$3,500 - $4,500 a month"
1,Teacher - Entry - Level to Experienced (Harlem...,Success Academy Charter Schools,NaN,"New York, NY",NaN
2,Part-Time Stocker,Aldi,3.3,"Brooklyn, NY",$19 an hour
3,Office Helper,CALL-A-HEAD CORP.,4.5,"Broad Channel, NY","$700 - $1,100 a week"
5,Part Time Cashier Associates,Lowe's,3.5,"Brooklyn, NY","$2,500 - $3,500 a month"
...,...,...,...,...,...
15099,General Bookkeeper,"Algin Management Co., LLC",NaN,"New York, NY",NaN
15100,Barista in Training,Interlude Coffee & Tea,NaN,"New York, NY",NaN
15101,"Hollister Co. - Stock Associate, FIFTH AVENUE",Hollister Co. Stores,3.5,"New York, NY +8 locations",NaN
15102,Part Time Branch Ambassador- King's Plaza,Capital One,3.9,"Brooklyn, NY",NaN


In [24]:
df.to_csv(r"C:\Users\Alif\Desktop\Python files\vscode_python\30-days-python\python_webscraping\raw_data_without_dups.csv", index=False)

In [31]:
df

,Title,Company,CompanyRating,Location,Salary
0,Warehouse Associate - Part Time (Days Shift),Lowe's,3.5,"New York, NY","$3,500 - $4,500 a month"
1,Teacher - Entry - Level to Experienced (Harlem...,Success Academy Charter Schools,NaN,"New York, NY",NaN
2,Part-Time Stocker,Aldi,3.3,"Brooklyn, NY",$19 an hour
3,Office Helper,CALL-A-HEAD CORP.,4.5,"Broad Channel, NY","$700 - $1,100 a week"
5,Part Time Cashier Associates,Lowe's,3.5,"Brooklyn, NY","$2,500 - $3,500 a month"
...,...,...,...,...,...
15099,General Bookkeeper,"Algin Management Co., LLC",NaN,"New York, NY",NaN
15100,Barista in Training,Interlude Coffee & Tea,NaN,"New York, NY",NaN
15101,"Hollister Co. - Stock Associate, FIFTH AVENUE",Hollister Co. Stores,3.5,"New York, NY +8 locations",NaN
15102,Part Time Branch Ambassador- King's Plaza,Capital One,3.9,"Brooklyn, NY",NaN


In [41]:
def formatter(i):    
    try:
        i = i.replace(',','')
    except:
        return np.nan
    if 'week' in i:
        i = re.search(r'\d+', i ).group()
        return (int(i)*54)
    elif 'day' in i:
        i = re.search(r'\d+', i ).group()
        return (int(i)*270)
    elif 'month' in i:
        i = re.search(r'\d+', i ).group()
        return (int(i)*12)
    elif 'year' in i:
        i = re.search(r'\d', i).group()
        return (int(i))
    else:
        i = re.search(r'\d+', i ).group()
        return (int(i)*2160)

In [42]:
df['Salary'] = df['Salary'].apply(formatter)

In [43]:
df

,Title,Company,CompanyRating,Location,Salary
0,Warehouse Associate - Part Time (Days Shift),Lowe's,3.5,"New York, NY",42000.0
1,Teacher - Entry - Level to Experienced (Harlem...,Success Academy Charter Schools,NaN,"New York, NY",NaN
2,Part-Time Stocker,Aldi,3.3,"Brooklyn, NY",41040.0
3,Office Helper,CALL-A-HEAD CORP.,4.5,"Broad Channel, NY",37800.0
5,Part Time Cashier Associates,Lowe's,3.5,"Brooklyn, NY",30000.0
...,...,...,...,...,...
15099,General Bookkeeper,"Algin Management Co., LLC",NaN,"New York, NY",NaN
15100,Barista in Training,Interlude Coffee & Tea,NaN,"New York, NY",NaN
15101,"Hollister Co. - Stock Associate, FIFTH AVENUE",Hollister Co. Stores,3.5,"New York, NY +8 locations",NaN
15102,Part Time Branch Ambassador- King's Plaza,Capital One,3.9,"Brooklyn, NY",NaN


In [45]:
df.to_csv(r"C:\Users\Alif\Desktop\Python files\vscode_python\web_projects\jobs_webscraping\final_data.csv", index=False)

In [56]:
# helpful video https://www.youtube.com/watch?v=CZlPxdSP1N8
Server = 'ALIFS-PC\SQLEXPRESS'
Database = 'Portfolio'
Driver = 'ODBC Driver 17 for SQL Server'
engine = sqlalchemy.create_engine(f'mssql://@{Server}/{Database}?driver={Driver}')

In [60]:
df.to_sql(
    name='Jobs_Webscraper',
    con = engine,
    index = False,
    if_exists='replace'
)

In [62]:
query = '''
SELECT TOP(10)
	[Title],
	COUNT(*) AS [Counts]
FROM [Portfolio].[dbo].[Jobs_Webscraper]
GROUP BY [Title] 
ORDER BY 2 DESC

'''

top_10_repeats = pd.read_sql_query(query, engine)
top_10_repeats

,Title,Counts
0,Front Desk Receptionist,16
1,Administrative Assistant,14
2,Sales Associate,14
3,Executive Assistant,13
4,server,13
5,Crew Member,12
6,Front Desk Agent,11
7,Warehouse Associate,10
8,Medical Receptionist,9
9,Receptionist,9
